In [2]:
#Load the required packages
import numpy as np
import pandas as pd
import multiprocessing as mp
import time

from subprocess import check_output
import subprocess
files=check_output(["ls", "/Users/yiming/Documents/BigData/final_project/churn_prediction/input"]).decode("utf8")
print(files)
#check the number of cores
num_cores = mp.cpu_count()
print("This kernel has :",num_cores,"cores")
#Check the number of row of each file
for file in files.split("\n"):
    path='/Users/yiming/Documents/BigData/final_project/churn_prediction/input/'+file
    popenobj=subprocess.Popen(['wc', '-l', path], stdout=subprocess.PIPE, 
                                              stderr=subprocess.PIPE)
    result,error= popenobj.communicate()
    print(result,error)
    if file != '':
        print("The file :",file,"has :",result.strip().split()[0],"rows")



members_v3.csv
sample_submission_zero.csv
train.csv
transactions.csv
user_logs.csv

('This kernel has :', 8, 'cores')
(' 6769474 /Users/yiming/Documents/BigData/final_project/churn_prediction/input/members_v3.csv\n', '')
('The file :', u'members_v3.csv', 'has :', '6769474', 'rows')
('  970961 /Users/yiming/Documents/BigData/final_project/churn_prediction/input/sample_submission_zero.csv\n', '')
('The file :', u'sample_submission_zero.csv', 'has :', '970961', 'rows')
('  992932 /Users/yiming/Documents/BigData/final_project/churn_prediction/input/train.csv\n', '')
('The file :', u'train.csv', 'has :', '992932', 'rows')
(' 21547747 /Users/yiming/Documents/BigData/final_project/churn_prediction/input/transactions.csv\n', '')
('The file :', u'transactions.csv', 'has :', '21547747', 'rows')
(' 392106544 /Users/yiming/Documents/BigData/final_project/churn_prediction/input/user_logs.csv\n', '')
('The file :', u'user_logs.csv', 'has :', '392106544', 'rows')
('', 'wc: /Users/yiming/Documents/Big

In [4]:
# testing out the funciton for one iteration
import time
size=1e6   # 1 million
user_log_chunks = pd.read_csv('/Users/yiming/Documents/BigData/final_project/churn_prediction/input/user_logs.csv',chunksize=size,index_col=['msno'])
start_time = time.time()
for i,user_log_chunk in enumerate(user_log_chunks):
    #print(user_log_chunk.head())
    grouped_object=user_log_chunk.groupby(user_log_chunk.index,sort=False) # not sorting results in a minor speedup
    func = {'date':['min','max'], 
           'num_25':['sum'],'num_50':['sum'],
           'num_75':['sum'],'num_985':['sum'],
           'num_100':['sum'],'num_unq':['sum'],'total_secs':['sum']}
    if(i==0):  # for the fiest run create the result 
        result=grouped_object.agg(func)
        print(len(result))
        print("Round ",i,"took %s seconds" % (time.time() - start_time))
    else:
        result=result.append(grouped_object.agg(func))
        print(len(result))
        print("Round ",i,"took %s seconds" % (time.time() - start_time))
        break

202510
('Round ', 0, 'took 1.747631073 seconds')
407351
('Round ', 1, 'took 3.42227315903 seconds')


In [5]:
def process_user_log(chunk):
    grouped_object=chunk.groupby(chunk.index,sort=False) # not sorting results in a minor speedup
    func = {'date':['min','max','count'], 
           'num_25':['sum'],'num_50':['sum'],
           'num_75':['sum'],'num_985':['sum'],
           'num_100':['sum'],'num_unq':['sum'],'total_secs':['sum']}
    answer=grouped_object.agg(func)
    return answer
print("done")

done


In [8]:
size=4e7 # 40 million
reader = pd.read_csv('/Users/yiming/Documents/BigData/final_project/churn_prediction/input/user_logs.csv',chunksize=size,index_col=['msno'])
start_time = time.time()
for i in range(10):
    user_log_chunk=next(reader)
    if(i==0):
        result=process_user_log(user_log_chunk)
        print("Loop ",i,"took %s seconds" % (time.time() - start_time))
    else:
        result=result.append(process_user_log(user_log_chunk))
        print("Loop ",i,"took %s seconds" % (time.time() - start_time))
    del(user_log_chunk)   
    
print "done!"

('Loop ', 0, 'took 85.5681848526 seconds')
('Loop ', 1, 'took 173.99387598 seconds')
('Loop ', 2, 'took 265.079565048 seconds')
('Loop ', 3, 'took 358.512262821 seconds')
('Loop ', 4, 'took 454.040303946 seconds')
('Loop ', 5, 'took 547.411473036 seconds')
('Loop ', 6, 'took 642.989202976 seconds')
('Loop ', 7, 'took 742.241251945 seconds')
('Loop ', 8, 'took 845.753796816 seconds')
('Loop ', 9, 'took 935.215692997 seconds')
done!


In [9]:
print(len(result))
check=result.index.unique()
print(len(check))

19166671
5234111


In [10]:
#result.columns.values
result.columns = ['_'.join(col).strip() for col in result.columns.values]
result.columns

Index([u'num_985_sum', u'num_50_sum', u'total_secs_sum', u'date_min',
       u'date_max', u'date_count', u'num_unq_sum', u'num_100_sum',
       u'num_75_sum', u'num_25_sum'],
      dtype='object')

In [11]:
result.head()
func = {'date_min':['min'],'date_max':['max'],'date_count':['count'] ,
           'num_25_sum':['sum'],'num_50_sum':['sum'],
           'num_75_sum':['sum'],'num_985_sum':['sum'],
           'num_100_sum':['sum'],'num_unq_sum':['sum'],'total_secs_sum':['sum']}
processed_user_log=result.groupby(result.index).agg(func)
print(len(processed_user_log))
processed_user_log.columns = processed_user_log.columns.get_level_values(0)
print(processed_user_log.head())

5234111
                                              num_75_sum  date_min  \
msno                                                                 
+++4vcS9aMH7KWdfh5git6nA5fC5jjisd5H/NcM++WM=           0  20150427   
+++EI4HgyhgcJHIPXk/VRP7bt17+2joG39T6oEfJ+tc=           0  20160420   
+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=          14  20160909   
+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=         227  20150101   
+++TipL0Kt3JvgNE9ahuJ8o+drJAnQINtxD4c5GePXI=           2  20151230   

                                              date_count  num_unq_sum  \
msno                                                                    
+++4vcS9aMH7KWdfh5git6nA5fC5jjisd5H/NcM++WM=           1            2   
+++EI4HgyhgcJHIPXk/VRP7bt17+2joG39T6oEfJ+tc=           1            1   
+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=           3          179   
+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=          10        53604   
+++TipL0Kt3JvgNE9ahuJ8o+drJAnQINtxD4c5GePXI=           1       

In [12]:
processed_user_log.head()

,num_75_sum,date_min,date_count,num_unq_sum,total_secs_sum,num_50_sum,num_100_sum,num_25_sum,num_985_sum,date_max
msno,,,,,,,,,,
+++4vcS9aMH7KWdfh5git6nA5fC5jjisd5H/NcM++WM=,0,20150427,1,2,9.741100e+01,1,0,1,0,20150427
+++EI4HgyhgcJHIPXk/VRP7bt17+2joG39T6oEfJ+tc=,0,20160420,1,1,5.686800e+01,0,0,2,0,20160420
+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,14,20160909,3,179,4.999677e+04,12,171,60,7,20160915
+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,227,20150101,10,53604,1.466484e+07,249,59354,817,195,20170227
+++TipL0Kt3JvgNE9ahuJ8o+drJAnQINtxD4c5GePXI=,2,20151230,1,22,3.661527e+03,3,14,3,1,20151230


In [8]:
# this function did not work because apparently only lists can be passed back from a function through the 
# multiprocessing package 
# If we use multiple cores the process can be 16 times faster

# create as many processes as there are CPUs on your machine
# if __name__ == '__main__':
#     size=1000000
#     parser = pd.read_csv('../input/user_logs.csv',chunksize=size,index_col=['msno'])
#     num_processes = mp.cpu_count()
#     pool=mp.Pool(processes=num_processes)
#     start_time = time.time()
#     list_of_ans=[]
#     def collect_results(result):
#         list_of_ans.extend(result)
#         print(done)
#     for i,df in enumerate(parser):
#         print("starting at",i," %s seconds" % (time.time() - start_time))
#         pool.map(process_user_log,df,callback=collect_results) 
#         
#     print("action ",i,"took %s seconds" % (time.time() - start_time))
#     pool.close()
#     pool.join()

<type 'numpy.ndarray'>


In [14]:
print "start"
processed_user_log.to_csv("processed_user_log.csv")
print "end"

start
end
